In [1]:
%%python
from importlib.util import find_spec
import shutil
import subprocess

fix = """
-------------------------------------------------------------------------
To fix this, follow the steps in the link below:
    https://github.com/modularml/mojo/issues/1085#issuecomment-1771403719
-------------------------------------------------------------------------
"""

def install_if_missing(name: str):
    if find_spec(name.replace("-", "_")):
        return

    print(f"The package `{name}` was not found. We will install it...")
    try:
        if shutil.which("python3"): python = "python3"
        elif shutil.which("python"): python = "python"
        else:
            raise RuntimeError("Python is not on `PATH`. " + fix)
        subprocess.check_call([python, "-m", "pip", "install", name])
    except:
        raise ImportError(f"The package `{name}` was not found. " + fix)

install_if_missing("mpmath")
install_if_missing("numpy")
install_if_missing("tabulate")


In [2]:
%%python
import mpmath as mp
import numpy as np


def py_numpy_polyval(coeffs, x):
    return np.polyval(coeffs[::-1], x)


def py_mpmath_polyval(coeffs, x):
    _coeffs = [mp.mpf(c) for c in coeffs[::-1]]

    def _mp_polyval_impl(a):
        with mp.workdps(350):
            res = mp.polyval(_coeffs, mp.mpf(a))
        return res

    dtype = np.result_type(x)
    return np.frompyfunc(_mp_polyval_impl, 1, 1)(x).astype(dtype)


In [3]:
import math

from python import Python
from python.object import PythonObject
from utils.static_tuple import StaticTuple

from specials._internal.limits import FloatLimits
from specials._internal.polynomial import Polynomial
from specials._internal.tensor import (
    elementwise,
    random_uniform,
    run_benchmark,
    tensor_to_numpy_array,
    UnaryOperator,
)

Python.add_to_path(".")


fn solution_report[
    solution_name: StringLiteral,
    func: UnaryOperator,
    dtype: DType,
    simd_width: Int = simdwidthof[dtype](),
](x: Tensor[dtype], truth: PythonObject) raises -> PythonObject:
    """Computes the evaluation metrics for a given numerical solution in Mojo."""
    let builtins = Python.import_module("builtins")
    let np = Python.import_module("numpy")
    let numerics_testing = Python.import_module("specials._internal.numerics_testing")

    let result = elementwise[func](x)
    let msecs = run_benchmark[func](x).mean("ms")
    let relerr = numerics_testing.py_relative_error(
        tensor_to_numpy_array(result), truth
    )

    let report = builtins.list()
    _ = report.append(solution_name)
    _ = report.append(np.max(relerr))
    _ = report.append(np.mean(relerr))
    _ = report.append(msecs)

    return report


In [4]:
%%python
from timeit import timeit

from specials._internal import numerics_testing


def py_benchmark(func, *args):
    """Computes the average execution time of a Python function."""
    # Warmup phase
    _ = timeit(lambda: func(*args), number=2)

    msecs = 1000 * timeit(lambda: func(*args), number=100) / 100
    return msecs


def py_solution_report(solution_name, func, coeffs, x_arr, truth):
    """Computes the evaluation metrics for a given numerical solution in Python."""
    result = func(coeffs, x_arr)
    msecs = py_benchmark(func, coeffs, x_arr)
    relerr = numerics_testing.py_relative_error(result, truth)

    return [solution_name, np.max(relerr), np.mean(relerr), msecs]


In [5]:
%%python
from tabulate import tabulate, SEPARATING_LINE


def py_print_table(
    data, domain_names, num_solutions, experiment_name
):
    """Prints the evaluation metrics for all numerical solutions."""
    headers = [
        "\nDegree",
        "\nSolution",
        "Maximum\nRelative Error",
        "Mean\nRelative Error",
        "Mean Execution Time\n(in milliseconds)",
    ]

    # Insert domain names
    current_domain = 0
    for i, report in enumerate(data):
        if i % num_solutions == 0:
            domain_name = domain_names[current_domain]
            data[i].insert(0, domain_name)
            current_domain += 1
        else:
            data[i].insert(0, "")

    # Insert horizontal lines between domains
    for index in range(num_solutions, len(data) + num_solutions, num_solutions + 1):
        data.insert(index, SEPARATING_LINE)

    print(f"\nExperiment: {experiment_name}\n")

    floatfmt = (".2e", ".2e", ".2e", ".2e", ".3f")
    table = tabulate(data, headers, tablefmt="simple", floatfmt=floatfmt)

    print(table)


In [6]:
alias dtype = DType.float32
alias simd_width = simdwidthof[dtype]()


In [7]:
random.seed(42)
let x = random_uniform[dtype](-10.0, 10.0, 250_000)
let x_arr = tensor_to_numpy_array(x)


In [8]:
let builtins = Python.import_module("builtins")
let domain_names = builtins.list()
let data = builtins.list()


### Degree 1

In [9]:
fn mojo_polyval_varlist(x: SIMD) -> SIMD[x.type, x.size]:
    return math.polynomial_evaluate[
        x.type,
        x.size,
        VariadicList(
            SIMD[x.type, x.size](1.0),
            SIMD[x.type, x.size](0.5),
        ),
    ](x)


In [10]:
fn mojo_polyval_tuple(x: SIMD) -> SIMD[x.type, x.size]:
    return math.polynomial_evaluate[
        x.type,
        x.size,
        2,
        StaticTuple[2, SIMD[x.type, x.size]](
            1.0,
            0.5,
        ),
    ](x)


In [11]:
fn specials_polyval(x: SIMD) -> SIMD[x.type, x.size]:
    alias p = Polynomial[2, x.type, x.size].from_coefficients[
        1.0,
        0.5,
    ]()
    return p(x)


In [12]:
%%python
coeffs = [
    1.0,
    0.5,
]


In [13]:
_ = domain_names.append("1")

# mpmath
let truth = py_mpmath_polyval(coeffs, x_arr)

# Specials
let specials_report = solution_report["Specials", specials_polyval, dtype](x, truth)
_ = data.append(specials_report)

# Mojo - VariadicList
let mojo_varlist_report = solution_report[
    "Mojo - VariadicList", mojo_polyval_varlist, dtype
](x, truth)
_ = data.append(mojo_varlist_report)

# Mojo - StaticTuple
let mojo_tuple_report = solution_report[
    "Mojo - StaticTuple", mojo_polyval_tuple, dtype
](x, truth)
_ = data.append(mojo_tuple_report)

# NumPy
let numpy_report = py_solution_report("NumPy", py_numpy_polyval, coeffs, x_arr, truth)
_ = data.append(numpy_report)


### Degree 2

In [14]:
fn mojo_polyval_varlist(x: SIMD) -> SIMD[x.type, x.size]:
    return math.polynomial_evaluate[
        x.type,
        x.size,
        VariadicList(
            SIMD[x.type, x.size](1.0),
            SIMD[x.type, x.size](0.5),
            SIMD[x.type, x.size](0.25),
        ),
    ](x)


In [15]:
fn mojo_polyval_tuple(x: SIMD) -> SIMD[x.type, x.size]:
    return math.polynomial_evaluate[
        x.type,
        x.size,
        3,
        StaticTuple[3, SIMD[x.type, x.size]](
            1.0,
            0.5,
            0.25,
        ),
    ](x)


In [16]:
fn specials_polyval(x: SIMD) -> SIMD[x.type, x.size]:
    alias p = Polynomial[3, x.type, x.size].from_coefficients[
        1.0,
        0.5,
        0.25,
    ]()
    return p(x)


In [17]:
%%python
coeffs = [
    1.0,
    0.5,
    0.25,
]


In [18]:
_ = domain_names.append("2")

# mpmath
let truth = py_mpmath_polyval(coeffs, x_arr)

# Specials
let specials_report = solution_report["Specials", specials_polyval, dtype](x, truth)
_ = data.append(specials_report)

# Mojo - VariadicList
let mojo_varlist_report = solution_report[
    "Mojo - VariadicList", mojo_polyval_varlist, dtype
](x, truth)
_ = data.append(mojo_varlist_report)

# Mojo - StaticTuple
let mojo_tuple_report = solution_report[
    "Mojo - StaticTuple", mojo_polyval_tuple, dtype
](x, truth)
_ = data.append(mojo_tuple_report)

# NumPy
let numpy_report = py_solution_report("NumPy", py_numpy_polyval, coeffs, x_arr, truth)
_ = data.append(numpy_report)


### Degree 4

In [19]:
fn mojo_polyval_varlist(x: SIMD) -> SIMD[x.type, x.size]:
    return math.polynomial_evaluate[
        x.type,
        x.size,
        VariadicList(
            SIMD[x.type, x.size](1.0),
            SIMD[x.type, x.size](0.5),
            SIMD[x.type, x.size](0.25),
            SIMD[x.type, x.size](0.125),
            SIMD[x.type, x.size](0.0625),
        ),
    ](x)


In [20]:
fn mojo_polyval_tuple(x: SIMD) -> SIMD[x.type, x.size]:
    return math.polynomial_evaluate[
        x.type,
        x.size,
        5,
        StaticTuple[5, SIMD[x.type, x.size]](
            1.0,
            0.5,
            0.25,
            0.125,
            0.0625,
        ),
    ](x)


In [21]:
fn specials_polyval(x: SIMD) -> SIMD[x.type, x.size]:
    alias p = Polynomial[5, x.type, x.size].from_coefficients[
        1.0,
        0.5,
        0.25,
        0.125,
        0.0625,
    ]()
    return p(x)


In [22]:
%%python
coeffs = [
    1.0,
    0.5,
    0.25,
    0.125,
    0.0625,
]


In [23]:
_ = domain_names.append("4")

# mpmath
let truth = py_mpmath_polyval(coeffs, x_arr)

# Specials
let specials_report = solution_report["Specials", specials_polyval, dtype](x, truth)
_ = data.append(specials_report)

# Mojo - VariadicList
let mojo_varlist_report = solution_report[
    "Mojo - VariadicList", mojo_polyval_varlist, dtype
](x, truth)
_ = data.append(mojo_varlist_report)

# Mojo - StaticTuple
let mojo_tuple_report = solution_report[
    "Mojo - StaticTuple", mojo_polyval_tuple, dtype
](x, truth)
_ = data.append(mojo_tuple_report)

# NumPy
let numpy_report = py_solution_report("NumPy", py_numpy_polyval, coeffs, x_arr, truth)
_ = data.append(numpy_report)


### Degree 8

In [24]:
fn mojo_polyval_varlist(x: SIMD) -> SIMD[x.type, x.size]:
    return math.polynomial_evaluate[
        x.type,
        x.size,
        VariadicList(
            SIMD[x.type, x.size](1.0),
            SIMD[x.type, x.size](0.5),
            SIMD[x.type, x.size](0.25),
            SIMD[x.type, x.size](0.125),
            SIMD[x.type, x.size](0.0625),
            SIMD[x.type, x.size](0.03125),
            SIMD[x.type, x.size](0.015625),
            SIMD[x.type, x.size](0.0078125),
            SIMD[x.type, x.size](0.00390625),
        ),
    ](x)


In [25]:
fn mojo_polyval_tuple(x: SIMD) -> SIMD[x.type, x.size]:
    return math.polynomial_evaluate[
        x.type,
        x.size,
        9,
        StaticTuple[9, SIMD[x.type, x.size]](
            1.0,
            0.5,
            0.25,
            0.125,
            0.0625,
            0.03125,
            0.015625,
            0.0078125,
            0.00390625,
        ),
    ](x)


In [26]:
fn specials_polyval(x: SIMD) -> SIMD[x.type, x.size]:
    alias p = Polynomial[9, x.type, x.size].from_coefficients[
        1.0,
        0.5,
        0.25,
        0.125,
        0.0625,
        0.03125,
        0.015625,
        0.0078125,
        0.00390625,
    ]()
    return p(x)


In [27]:
%%python
coeffs = [
    1.0,
    0.5,
    0.25,
    0.125,
    0.0625,
    0.03125,
    0.015625,
    0.0078125,
    0.00390625,
]


In [28]:
_ = domain_names.append("8")

# mpmath
let truth = py_mpmath_polyval(coeffs, x_arr)

# Specials
let specials_report = solution_report["Specials", specials_polyval, dtype](x, truth)
_ = data.append(specials_report)

# Mojo - VariadicList
let mojo_varlist_report = solution_report[
    "Mojo - VariadicList", mojo_polyval_varlist, dtype
](x, truth)
_ = data.append(mojo_varlist_report)

# Mojo - StaticTuple
let mojo_tuple_report = solution_report[
    "Mojo - StaticTuple", mojo_polyval_tuple, dtype
](x, truth)
_ = data.append(mojo_tuple_report)

# NumPy
let numpy_report = py_solution_report("NumPy", py_numpy_polyval, coeffs, x_arr, truth)
_ = data.append(numpy_report)


### Degree 16

In [29]:
fn mojo_polyval_varlist(x: SIMD) -> SIMD[x.type, x.size]:
    return math.polynomial_evaluate[
        x.type,
        x.size,
        VariadicList(
            SIMD[x.type, x.size](1.0),
            SIMD[x.type, x.size](0.5),
            SIMD[x.type, x.size](0.25),
            SIMD[x.type, x.size](0.125),
            SIMD[x.type, x.size](0.0625),
            SIMD[x.type, x.size](0.03125),
            SIMD[x.type, x.size](0.015625),
            SIMD[x.type, x.size](0.0078125),
            SIMD[x.type, x.size](0.00390625),
            SIMD[x.type, x.size](0.001953125),
            SIMD[x.type, x.size](0.0009765625),
            SIMD[x.type, x.size](0.00048828125),
            SIMD[x.type, x.size](0.000244140625),
            SIMD[x.type, x.size](0.0001220703125),
            SIMD[x.type, x.size](6.103515625e-05),
            SIMD[x.type, x.size](3.0517578125e-05),
            SIMD[x.type, x.size](1.52587890625e-05),
        ),
    ](x)


In [30]:
fn mojo_polyval_tuple(x: SIMD) -> SIMD[x.type, x.size]:
    return math.polynomial_evaluate[
        x.type,
        x.size,
        17,
        StaticTuple[17, SIMD[x.type, x.size]](
            1.0,
            0.5,
            0.25,
            0.125,
            0.0625,
            0.03125,
            0.015625,
            0.0078125,
            0.00390625,
            0.001953125,
            0.0009765625,
            0.00048828125,
            0.000244140625,
            0.0001220703125,
            6.103515625e-05,
            3.0517578125e-05,
            1.52587890625e-05,
        ),
    ](x)


In [31]:
fn specials_polyval(x: SIMD) -> SIMD[x.type, x.size]:
    alias p = Polynomial[17, x.type, x.size].from_coefficients[
        1.0,
        0.5,
        0.25,
        0.125,
        0.0625,
        0.03125,
        0.015625,
        0.0078125,
        0.00390625,
        0.001953125,
        0.0009765625,
        0.00048828125,
        0.000244140625,
        0.0001220703125,
        6.103515625e-05,
        3.0517578125e-05,
        1.52587890625e-05,
    ]()
    return p(x)


In [32]:
%%python
coeffs = [
    1.0,
    0.5,
    0.25,
    0.125,
    0.0625,
    0.03125,
    0.015625,
    0.0078125,
    0.00390625,
    0.001953125,
    0.0009765625,
    0.00048828125,
    0.000244140625,
    0.0001220703125,
    6.103515625e-05,
    3.0517578125e-05,
    1.52587890625e-05,
]


In [33]:
_ = domain_names.append("16")

# mpmath
let truth = py_mpmath_polyval(coeffs, x_arr)

# Specials
let specials_report = solution_report["Specials", specials_polyval, dtype](x, truth)
_ = data.append(specials_report)

# Mojo - VariadicList
let mojo_varlist_report = solution_report[
    "Mojo - VariadicList", mojo_polyval_varlist, dtype
](x, truth)
_ = data.append(mojo_varlist_report)

# Mojo - StaticTuple
let mojo_tuple_report = solution_report[
    "Mojo - StaticTuple", mojo_polyval_tuple, dtype
](x, truth)
_ = data.append(mojo_tuple_report)

# NumPy
let numpy_report = py_solution_report("NumPy", py_numpy_polyval, coeffs, x_arr, truth)
_ = data.append(numpy_report)


In [34]:
py_print_table(data, domain_names, 4, "Polynomal Evaluation" + " (" + str(dtype) + ")")



Experiment: Polynomal Evaluation (float32)

                                        Maximum              Mean    Mean Execution Time
Degree    Solution               Relative Error    Relative Error      (in milliseconds)
--------  -------------------  ----------------  ----------------  ---------------------
1         Specials                     0.00e+00          0.00e+00                  0.097
          Mojo - VariadicList          0.00e+00          0.00e+00                  0.099
          Mojo - StaticTuple           0.00e+00          0.00e+00                  0.098
          NumPy                        0.00e+00          0.00e+00                  0.430
--------  -------------------  ----------------  ----------------  ---------------------
2         Specials                     0.00e+00          0.00e+00                  0.102
          Mojo - VariadicList          1.19e-07          1.69e-08                  0.102
          Mojo - StaticTuple           1.19e-07          1.69e-08

## Appendix A: System information

Below, information about the system used to run the experiment.

In [35]:
%%python

subprocess.run(["modular", "-v"])
subprocess.run(["mojo", "-v"])


modular 0.4.1 (2d8afe15)
mojo 0.7.0 (af002202)


In [36]:
from sys.info import (
    os_is_linux,
    os_is_windows,
    os_is_macos,
    has_sse4,
    has_avx,
    has_avx2,
    has_avx512f,
    has_vnni,
    has_neon,
    is_apple_m1,
    has_intel_amx,
    num_physical_cores,
    _current_target,
    _current_cpu,
    _triple_attr,
)

let os: StringLiteral
if os_is_linux():
    os = "linux"
elif os_is_macos():
    os = "macOS"
else:
    os = "windows"

let cpu = String(_current_cpu())
let arch = String(_triple_attr())

var cpu_features = String("")
if has_sse4():
    cpu_features += " sse4"
if has_avx():
    cpu_features += " avx"
if has_avx2():
    cpu_features += " avx2"
if has_avx512f():
    cpu_features += " avx512f"
if has_vnni():
    if has_avx512f():
        cpu_features += " avx512_vnni"
    else:
        cpu_features += " avx_vnni"
if has_intel_amx():
    cpu_features += " intel_amx"
if has_neon():
    cpu_features += " neon"
if is_apple_m1():
    cpu_features += " apple_m1"

if len(cpu_features) > 0:
    cpu_features = cpu_features[1:]

print("System Information")
print("    OS          :", os)
print("    CPU         :", cpu)
print("    Arch        :", arch)
print("    Num Cores   :", num_physical_cores())
print("    CPU Features:", cpu_features)


System Information
    OS          : linux
    CPU         : haswell
    Arch        : x86_64-unknown-linux-gnu
    Num Cores   : 4
    CPU Features: sse4 avx avx2


In [37]:
%%python
import pkg_resources
import sys

def get_version(package):
    """Returns the version of a Python package."""
    return pkg_resources.get_distribution(package).version

print("mpmath version:", mp.__version__)
print("NumPy version:", np.__version__)
print("Python version:", sys.version)
print("Tabulate version:", get_version("tabulate"))


mpmath version: 1.3.0
NumPy version: 1.26.2
Python version: 3.11.5 (main, Sep 11 2023, 14:07:11) [GCC 11.2.0]
Tabulate version: 0.9.0
